In [1]:
import itertools as it
import string
import pandas as pd
from datetime import datetime

In [2]:
letters = list(string.ascii_uppercase)

In [3]:
numbers = list(map(str, (list(range(10)))))

In [4]:
postal_product = list(it.product(['V'], numbers, letters, numbers, letters, numbers))

In [5]:
postal_codes = list(map(''.join, postal_product))

In [6]:
df = pd.DataFrame(postal_codes, columns = ['Postal codes'])

In [7]:
df.to_csv('Work_tables/all_possible_postal_code.csv', index = False)

In [8]:
pcdf = pd.read_csv('Work_tables/gcs_results.csv').dropna(subset=['ACTIVE'])

In [9]:
pcdf = pcdf.loc[pcdf['ACTIVE'] == 'Y']

In [10]:
pcdf = pcdf.drop('ACTIVE', axis=1)

In [11]:
excel_lookup = pd.ExcelFile('Work_tables/GCS_Lookup_Table.xlsx')

In [12]:
sheet_names = excel_lookup.sheet_names

In [13]:
common_sheets = list(set(pcdf.columns).intersection(sheet_names))

In [14]:
lookup_dict = {}

In [15]:
pcdf.columns

Index(['POSTALCODE', 'BIRTH_DATE', 'RET_DATE', 'LATITUDE', 'LONGITUDE',
       'COMM_NAME', 'CD_2021', 'CSD_2021', 'CDCSD_2021', 'MUN_NAME_2021',
       'CMACA_2021', 'CT_2021', 'DA_2021', 'DPL_2021', 'DR_2021',
       'POPCTR_2021'],
      dtype='object')

In [16]:
for s in common_sheets:

    lookup_dict[s] =  {'file': pd.read_excel('Work_tables/GCS_Lookup_Table.xlsx', sheet_name = s)}
    lookup_dict[s]['key'] = ''.join(set(lookup_dict[s]['file'].columns.to_list()).intersection(set(pcdf.columns.to_list())))
    lookup_dict[s]['values'] = lookup_dict[s]['file'].columns.to_list()
    lookup_dict[s]['values'].remove(lookup_dict[s]['key'])
    numeric_lookup_cols = lookup_dict[s]['file'].select_dtypes(include='number').columns.tolist()
    lookup_dict[s]['file'][numeric_lookup_cols] = lookup_dict[s]['file'][numeric_lookup_cols].astype('float')

In [17]:
for sheet in common_sheets:
    pcdf = pcdf.merge(lookup_dict[sheet]['file'], how = 'left')

In [18]:
for sheet in common_sheets:
    for i in range(len(lookup_dict[sheet]['values'])):

        no_match_count = (pcdf[lookup_dict[sheet]['file']
             .columns.to_list()]
             .loc[(pcdf[lookup_dict[sheet]['key']].isna() == False) & pcdf[lookup_dict[sheet]['values'][i]]
             .isna()]
             .shape[0]
            )
        
        if no_match_count != 0:
            print(f"{sheet}: {lookup_dict[sheet]['values'][i]}. Count: {no_match_count}")
        else:
            print(f"{sheet}: {lookup_dict[sheet]['values'][i]}. No issues")

DPL_2021: DPLNAME. No issues
CMACA_2021: CMANAME. No issues
POPCTR_2021: PCNAME. No issues
CD_2021: CDNAME. No issues
CDCSD_2021: CDCSDNAME. No issues
CDCSD_2021: CDCSDTYPE. No issues
DR_2021: DRNAME. No issues


In [19]:
for sheet in lookup_dict:
    new_loc = pcdf.columns.get_loc(lookup_dict[sheet]['key']) + 1
    for i in range(len(lookup_dict[sheet]['values'])):
        col_to_move = pcdf.pop(lookup_dict[sheet]['values'][i])
        pcdf.insert(new_loc, lookup_dict[sheet]['values'][i], col_to_move)

In [20]:
pcdf.columns

Index(['POSTALCODE', 'BIRTH_DATE', 'RET_DATE', 'LATITUDE', 'LONGITUDE',
       'COMM_NAME', 'CD_2021', 'CDNAME', 'CSD_2021', 'CDCSD_2021', 'CDCSDTYPE',
       'CDCSDNAME', 'MUN_NAME_2021', 'CMACA_2021', 'CMANAME', 'CT_2021',
       'DA_2021', 'DPL_2021', 'DPLNAME', 'DR_2021', 'DRNAME', 'POPCTR_2021',
       'PCNAME'],
      dtype='object')

In [21]:
pcdf.to_excel(''.join(
    ['_'.join(
        ['Geocoded_postal_codes/Geocoded_postal_codes', 
         datetime.today().strftime('%Y_%m_%d')]
    ), '.xlsx']),
    index=False
    )